In [2]:
import kagglehub
import pandas as pd
import json
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch


/home/rafal/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/rafal/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


### Import Data


In [3]:
path = kagglehub.dataset_download("mathurinache/math-dataset")

In [4]:

problems = {"problem": [], "level": [], "type": [], "solution": []}

with os.scandir(path + "/MATH/train/") as types:
    for type in types:
        if type.is_dir():
            with os.scandir(type) as entries:
                for entry in entries:
                    if entry.is_file():
                        # print(entry.path)
                        with open(entry.path) as f:
                            data = json.load(f)
                            for key in data:
                                problems[key].append(data[key])

all_levels = pd.DataFrame.from_dict(problems, orient='columns')

# BLOCK_SIZE = max(all_levels["problem"].apply(len) + all_levels["solution"].apply(len))
# BLOCK_SIZE

#### Split data

In [5]:
train, test = train_test_split(all_levels, test_size=0.15, random_state=42, stratify=all_levels['level'])

print(len(train[train['level'] == "Level 1"]), '/', len(train))
print(len(test[test['level'] == "Level 1"]), '/', len(test))

479 / 6375
85 / 1125


### Model Training

In [11]:
from mingpt.model import GPT
model_config = GPT.get_default_config()
model_config.model_type = 'gpt-micro'
model_config.vocab_size = 50257 # openai's model vocabulary
model_config.block_size = 1024  # openai's model block_size (i.e. input context length)
model = GPT(model_config)

BLOCK_SIZE = model_config.block_size

number of parameters: 7.36M


#### Create dataset

In [12]:
from mingpt.bpe import BPETokenizer

tokenizer = BPETokenizer()

class MathDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def get_block_size(self):
        return BLOCK_SIZE

    def __getitem__(self, idx):
        problem = self.data.iloc[idx]['problem']
        solution = self.data.iloc[idx]['solution']
        input = tokenizer(problem + ' ' + solution)
        x = input.clone().detach()[0][:-1]
        y = input.clone().detach()[0][1:]
        
        if len(x) > BLOCK_SIZE:
            return self.__getitem__(idx + 1)
        
        if len(x) < BLOCK_SIZE:
            x = torch.cat((x, torch.zeros(BLOCK_SIZE - len(x)).long()))
            y = torch.cat((y, torch.zeros(BLOCK_SIZE - len(y)).long()))

        

        y[:len(problem)] = -1
        # print(x, y)
        return x, y
        
    
train_dataset = MathDataset(train)
test_dataset = MathDataset(test)

train_dataset.__getitem__(0)

(tensor([2437,  867, 3967,  ...,    0,    0,    0]),
 tensor([-1, -1, -1,  ...,  0,  0,  0]))

In [14]:
from mingpt.trainer import Trainer
train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # many possible options, see the file
train_config.max_iters = 1000
train_config.batch_size = 32
trainer = Trainer(train_config, model, train_dataset)
trainer.run()

running on device cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 9.99 GiB is allocated by PyTorch, and 524.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)